In [272]:
#pip install azureml-sdk



In [273]:
import os
import pandas as pd
import logging
import json
from azureml.core import Workspace, Dataset, Experiment
from azureml.train.automl import AutoMLConfig
from azureml.core.environment import Environment
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.automl.core.forecasting_parameters import ForecastingParameters
from azureml.train.automl import AutoMLConfig
from azureml.core import Dataset, Experiment

In [274]:
# Connect to the Azure ML workspace
ws = Workspace.from_config()

# Get the default datastore
datastore = ws.get_default_datastore()

# Get compute_name from config.json
# Load the settings from config.json
with open("./config.json") as f:
    config = json.load(f)

compute_name = config["compute_name"]

# Retrieve or create a compute cluster
if compute_name not in ws.compute_targets:
    # If the cluster does not exist, create a new one
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",  # VM size
                                                           min_nodes=0,  # Minimum number of nodes
                                                           max_nodes=4)  # Maximum number of nodes

    # Create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, compute_config)

    # Wait until the cluster is created
    compute_target.wait_for_completion(show_output=True)
else:
    # Retrieve the existing cluster
    compute_target = ws.compute_targets[compute_name]

# Check if the cluster was successfully retrieved or created
print(f"Compute target: {compute_target.name}")

# Check the status of the cluster
print(compute_target.get_status().serialize())


Compute target: cluster-001
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 1, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2024-09-18T00:46:58.563000+00:00', 'errors': None, 'creationTime': '2024-09-17T02:16:12.974644+00:00', 'modifiedTime': '2024-09-18T15:47:23.658386+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 1, 'maxNodeCount': 3, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'Standard_E4ds_v4'}


In [275]:
compute_target.update(min_nodes=1, max_nodes=3)  # Adjust the number of nodes
compute_target.wait_for_completion(show_output=True)  # Wait until completion

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [277]:
# データの前処理
data = pd.read_csv("./data/train_data.csv")

# Date列を日付型に変換
data['timestamp'] = pd.to_datetime(data['timestamp'])

# 欠損値の処理
data = data.fillna(method='ffill')

# デフォルトのデータストアにデータセットを登録
datastore = Datastore.get_default(ws)

# Pandasデータフレームをデータストアに登録
dataset = Dataset.Tabular.register_pandas_dataframe(data, datastore, "timeseries_dataset")

# ForecastingParameters の設定
forecasting_parameters = ForecastingParameters(
    time_column_name='timestamp',
    forecast_horizon=10,
    target_lags=None,
    rolling_window_size=None
)

# AutoML 設定
automl_config = AutoMLConfig(
    task='forecasting',
    primary_metric='normalized_root_mean_squared_error',
    experiment_timeout_minutes=30,
    training_data=dataset,
    label_column_name='Sales',
    n_cross_validations=5,
    enable_early_stopping=True,
    target_lags=None,
    rolling_window_size=None,
    compute_target=compute_target,
    forecasting_parameters=forecasting_parameters
)

# 実験の作成
experiment = Experiment(ws, 'automl-timeseries-forecasting')


Validating arguments.
Arguments validated.
Validating arguments.
Arguments validated.
'overwrite' is set to True. Any file already present in the target will be overwritten.
Uploading files from 'C:/Users/MAKITS~1/AppData/Local/Temp/tmp8opkztus' to 'managed-dataset/7d23f1ad-081b-44f6-a8b0-a25a0d04a8a1/'
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [278]:
# 実験の実行
local_run = experiment.submit(automl_config, show_output=True)

# 最良モデルの取得
best_run, fitted_model = local_run.get_output()

# 最良モデルの保存
best_run.register_model(model_name="best_timeseries_model", model_path="./outputs/model")

Submitting remote run.
No run_configuration provided, running on cluster-001 with default configuration
Running on remote compute: cluster-001


Experiment,Id,Type,Status,Details Page,Docs Page
automl-timeseries-forecasting,AutoML_5039c490-b909-41a2-95d0-0dc8aff3d5c7,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Time Series ID detection
STATUS:       PASSED
DESCRIPTION:  The data set was analyzed, and no duplicate time index were detected.
              Learn more about time-series forecasting configurations: https://aka.ms/AutomatedMLForecastingConfiguration

********************************************************************************************

TYPE:         Short series handling
STATUS:       PASSED
DESCRIPTION:  Automated ML detected enough data points for each series in the input data to continue with training.
              Learn more about short series handling: https://aka.ms/AutomatedMLShortSeriesHandling

********************************************************************************************

TYPE:         Frequency detection
STATUS:       PASSED
DESCRIPTION:  The time series was analyzed,

    0    Naive                                         0:00:09             0.0819    0.0819
    1    SeasonalNaive                                 0:00:08             0.0973    0.0819
